In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf

In [2]:
#fetching stock details

def fetch_stock_data(stock_url):
    response = requests.get(stock_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    data = {}
    try:
        data['PE'] = soup.find(string="Market Cap").find_next(class_="value").text.strip()
        data['EPS'] = soup.find(string="EPS").find_next(class_="value").text.strip()
        data['52_Week_High'] = soup.find(string="52 Week High").find_next(class_="value").text.strip()
        data['52_Week_Low'] = soup.find(string="52 Week Low").find_next(class_="value").text.strip()
        data['LTP'] = soup.find(string="LTP").find_next(class_="value").text.strip()
        data['Market_Cap'] = soup.find(string="Market Cap").find_next(class_="value").text.strip()
        data['Volume'] = soup.find(string="Volume").find_next(class_="value").text.strip()
        data['Change_Percentage'] = soup.find(string="Change").find_next(class_="value").text.strip()
    except AttributeError as e:
        print(f"Data missing for some fields: {e}")
    return data


In [3]:
#stock returns
def fetch_returns(stock_symbols):
    returns_data = []
    for symbol in stock_symbols:
        ticker = yf.Ticker(symbol + ".NS")
        history = ticker.history(period="5y")

        if history.empty:
            returns_data.append({'Stock': symbol, '6M_Return': None, '1Y_Return': None, '5Y_Return': None})
            continue

        close_prices = history['Close']
        returns = {
            '6M_Return': ((close_prices.iloc[-1] / close_prices.iloc[-126]) - 1) * 100 if len(close_prices) > 126 else None,
            '1Y_Return': ((close_prices.iloc[-1] / close_prices.iloc[-252]) - 1) * 100 if len(close_prices) > 252 else None,
            '5Y_Return': ((close_prices.iloc[-1] / close_prices.iloc[0]) - 1) * 100 if len(close_prices) > 0 else None
        }
        returns['Stock'] = symbol
        returns_data.append(returns)

    return pd.DataFrame(returns_data)

In [4]:
def main():
    nifty50_url = "https://www.screener.in/company/"
    stocks = ["RELIANCE", "TCS", "INFY"]

    stock_data = []
    for stock in stocks:
        stock_url = f"{nifty50_url}{stock}/"
        data = fetch_stock_data(stock_url)
        data['Stock'] = stock
        stock_data.append(data)
    stock_df = pd.DataFrame(stock_data)

    #returns data
    returns_df = fetch_returns(stocks)

    final_df = pd.merge(stock_df, returns_df, on='Stock', how='left')

    print("Final DataFrame:")
    print(returns_df)

if __name__ == "__main__":
    main()

Data missing for some fields: 'NoneType' object has no attribute 'find_next'
Data missing for some fields: 'NoneType' object has no attribute 'find_next'
Data missing for some fields: 'NoneType' object has no attribute 'find_next'
Final DataFrame:
   6M_Return  1Y_Return   5Y_Return     Stock
0 -12.539191   4.142005   81.653247  RELIANCE
1  15.911730  25.922391  136.459431       TCS
2  35.377283  39.130994  215.967256      INFY
